# **#Introduction : In this end-to-end  project.** 
## we have used cricket 2023 in Test matches  data to build insights on a best 11 players team that we can assemble from the earth that can go and play with aliens. 
## We creating data web scraping to collect data from espncricinfo website then we performed some data transformation and cleaning in pandas, followed by building dashboards in power bi.
### Steps: 
### step 1 - creating data web scraping to collect data.
### step 2 - performed some data transformation and cleaning in pandas.
### step 3 - building dashboards in power bi.

# step 1 - creating data web scraping to collect data. 

In [1]:
from bs4 import BeautifulSoup # Web scraping package
import requests # To pull urls
import pandas as pd # For working with a dataframe


## This is a function that generates a table of matches and creates three lists:
## 1. url_links - containing the url links for game cards
## 2. match_IDs - containing the ID match
## 3. match_names - containing the name of the match.

In [2]:
def Creating_df_matche(rows): 
    # Creating lists for each column
    tabel_match = []
    
    # Creating list for ulr and dic for matchID match
    url_link = []
    list_match_ID = []
    list_match = []
    
    # Iterate over each row and extract the data
    for row in rows:
        # Find all data each row
        columns = row.find_all("td")
        # Adding values to each list
        tabel_match.append({
            "match" : columns[0].text.strip() + " VS " + columns[1].text.strip(),
            "winer" : columns[2].text.strip(),
            "margin" : columns[3].text.strip(),
            "ground" : columns[4].text.strip(),
            "match_date" : columns[5].text.strip(),
            "match_ID" : columns[6].text.strip()
        })
        url_link.append("https://www.espncricinfo.com" + columns[6].find('a').get('href')) 
        list_match_ID.append(columns[6].text.strip())
        list_match.append(columns[0].text.strip() + " VS " + columns[1].text.strip())

    
    
    # Creating match_results_df
    tabel_match_df = pd.DataFrame(tabel_match) 

    return tabel_match_df ,url_link ,list_match_ID, list_match

    
    

## This function takes in an URL, match ID, and match name as inputs and generates two tables:
## 1. table_batting - containing details of players who batted in the match
## 2. table_player_batting - containing personal details of players who batted in the match

In [4]:
def Creating_df_tabel_batting(url, match_ID, match):
    # Specify the URL of the webpage you want to scrape
    # Send an HTTP GET request to the URL
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        # Finding all tables with a given suffix and we know they are found in index 0 and 1
        tables = soup.select('div > table.ci-scorecard-table')
        # In these tables there are empty rows according to the HTML code, 
        # so we filtered for rows that have at least 11 columns as we need.
        firstInningRows = list(filter(lambda tr: len(tr.find_all("td")) >= 8, tables[0].select('tbody > tr')))
        secondInningsRows = list(filter(lambda tr: len(tr.find_all("td")) >= 8, tables[1].select('tbody > tr')))


        # Creating two list, first for players URL and second for batting Summary
        list_url_plyear = [] 
        battingSummary = []
        # A loop that builds these lists from the first table on the site
        for index, element in enumerate(firstInningRows):
            tds = element.find_all('td')
            battingSummary.append({
                "match": match,
                "match_ID" : match_ID,
                "teamInnings": "Innings 1",
                "battingPos": index+1,
                "batsmanName": tds[0].find('a').text,
                "Out/Not Out": "Not Out" if tds[1].find('span') == None else "Out" , 
                "runs": tds[2].find('strong').text,
                "balls": tds[3].text,
                "4s": tds[5].text,
                "6s": tds[6].text,
                "SR": tds[7].text
            })
            list_url_plyear.append("https://www.espncricinfo.com" + tds[0].find('a').get('href'))
        
        # A loop that builds these lists from the second table on the site
        for index, element in enumerate(secondInningsRows):
            tds = element.find_all('td')
            battingSummary.append({
                "match": match,
                "match_ID" : match_ID,
                "teamInnings": "Innings 2",
                "battingPos": index+1,
                "batsmanName": tds[0].find('a').text,
                "Out/Not Out": "Not Out" if tds[1].find('span') == None else "Out" ,
                "runs": tds[2].find('strong').text,
                "balls": tds[3].text,
                "4s": tds[5].text,
                "6s": tds[6].text,
                "SR": tds[7].text
            })
            list_url_plyear.append("https://www.espncricinfo.com" + tds[0].find('a').get('href'))

            
        # Build a list of all the players with their personal details,
        # We send the list of links to the method and it returns a dictionary of players    
        tabel_palyer_dic = Creating_df_tabel_player(list_url_plyear)         
        
        tabel_player_df = pd.DataFrame(tabel_palyer_dic)      
        tabel_Batting = pd.DataFrame(battingSummary) 
        
        return tabel_Batting , tabel_player_df

## This function takes in an URL, match ID, and match name as inputs and generates two tables:
## 1. table_bowling - containing details of players who bowled in the match
## 2. table_player_bowling - containing personal details of players who bowled in the match 

In [6]:
def Creating_df_tabel_bowling(url,match_ID, match):
    # Specify the URL of the webpage you want to scrape
    # Send an HTTP GET request to the URL
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        # Finding all tables with a given suffix and we know they are found in index 1 and 3
        tables = soup.select('div > table.ds-table') 
        # In these tables there are empty rows according to the HTML code, 
        # so we filtered for rows that have at least 11 columns as we need.
        firstInningRows = list(filter(lambda tr: len(tr.find_all("td")) >= 11, tables[1].select('tbody > tr')))
        secondInningsRows = list(filter(lambda tr: len(tr.find_all("td")) >= 11, tables[3].select('tbody > tr'))) 

        # Creating two list, first for players URL and second for bowling Summary
        list_url_plyear = []    
        bowlingSummary = []
        # A loop that builds these lists from the first table on the site
        for index, element in enumerate(firstInningRows):
            tds = element.find_all('td')
            bowlingSummary.append({
                "match": match,
                "match_ID" : match_ID,
                "teamInnings": "Innings 1",
                "bowlerName": tds[0].find('a').text,
                "overs": tds[1].text,
                "maiden": tds[2].text,
                "runs": tds[3].text,
                "wickets": tds[4].text,
                "economy": tds[5].text,
                "0s": tds[6].text,
                "4s": tds[7].text,
                "6s": tds[8].text,
                "wides": tds[9].text,
                "noBalls": tds[10].text
            })
            list_url_plyear.append("https://www.espncricinfo.com" + tds[0].find('a').get('href'))

        # A loop that builds these lists from the second table on the site    
        for index, element in enumerate(secondInningsRows):
            tds = element.find_all('td')
            bowlingSummary.append({
                "match": match,
                "match_ID" : match_ID,
                "teamInnings": "Innings 2",
                "bowlerName": tds[0].find('a').text,
                "overs": tds[1].text,
                "maiden": tds[2].text,
                "runs": tds[3].text,
                "wickets": tds[4].text,
                "economy": tds[5].text,
                "0s": tds[6].text,
                "4s": tds[7].text,
                "6s": tds[8].text,
                "wides": tds[9].text,
                "noBalls": tds[10].text
            })
            list_url_plyear.append("https://www.espncricinfo.com" + tds[0].find('a').get('href'))

            
        # Build a list of all the players with their personal details,
        # We send the list of links to the method and it returns a dictionary of players   
        tabel_palyer_dic = Creating_df_tabel_player(list_url_plyear)  
        
        
        tabel_player_df = pd.DataFrame(tabel_palyer_dic)   
        table_bowling = pd.DataFrame(bowlingSummary)
        
        return table_bowling, tabel_player_df

## This is a Python function that receives a list of URLs to player websites and builds a tabel of all the players' details.

In [7]:
def  Creating_df_tabel_player(list_url_player):
    # Creating a list contin info for players. 
    player_info = [] 
    
    # This is a loop that runs along all the web links of players and builds a list 
    for url in list_url_player:
        response = requests.get(url)
        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser') 

            # info from balck card in website   
            info_balck_card = soup.select('div > div.ds-p-0')[0]
            # info from white card in website
            info_white_card = soup.select('div > div.ds-p-4')[0]

            # x contin info team and role, 
            x = info_balck_card.find_all('span')
            # y contion info batting Style and bowling Style
            y = info_white_card.find_all('p')    

            player_info.append({
                "name" :info_balck_card.find('h1').text ,
                "team" : x[0].text ,
                "playingRole" : x[2].text,
                "battingStyle" : y[7].text ,
                "bowlingStyle" : y[9].text
            })  

    return player_info

## This is a Python function that receives a URL to a website with a summarized match table and returns four dataframes:
## 1. Match table
## 2. Batting table
## 3. Bowling table
## 4. Player table

In [8]:
# Specify the URL of the webpage you want to scrape
url = "https://www.espncricinfo.com/records/year/team-match-results/2023-2023/test-matches-1"

# Send an HTTP GET request to the URL
response = requests.get(url)
if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    # Find the table containing the match results
    table = soup.find("table")

    # Find all rows in the table except the header row
    rows = table.find_all("tr")[1:]

    # create a table Matche results 
    table_match_results, url_link ,list_match_ID, list_match  = Creating_df_matche(rows)

    # Creating a table of batting A table detailing players who batting the first link
    table_batting, table_player_batting = Creating_df_tabel_batting(url_link[0],list_match_ID[0],list_match[0])
    
    # Creating a table of bowling A table detailing players who bowling the first link
    table_bowling, table_player_bowling = Creating_df_tabel_bowling(url_link[0],list_match_ID[0],list_match[0])
    
    # A vertical connection of details of players who batting and who bowling.
    table_player = pd.concat([table_player_batting,table_player_bowling],ignore_index = True )

    # A loop that connects all the tables according to the url link to match scorecard
    for x in range(1,len(url_link)):
        # Creating a table of batting A table detailing players who batting the other links
        table_batting_temp , table_player_batting_temp = Creating_df_tabel_batting(url_link[x],list_match_ID[x],list_match[x])
        # Creating a table of bowling A table detailing players who bowling the other links
        table_bowling_temp , table_player_bowling_temp = Creating_df_tabel_bowling(url_link[x],list_match_ID[x],list_match[x])
        
        # Vertical connection of tables with batting
        table_batting = pd.concat([table_batting, table_batting_temp], ignore_index = True)
        # Vertical connection of tables with bowling 
        table_bowling = pd.concat([table_bowling, table_bowling_temp], ignore_index = True)
        # Vertical connection of tables with details of players who batting and who bowling
        table_player = pd.concat([table_player, table_player_batting_temp, table_player_bowling_temp], ignore_index = True) 
        
    # Creating CSV files Go to step 2 of the project    
    file_name1 = 'datasets/table_match_results.csv'
    file_name2 = 'datasets/table_batting.csv' 
    file_name3 = 'datasets/table_bowling.csv'
    file_name4 = 'datasets/table_player.csv' 

    table_match_results.to_csv(file_name1, index=False)
    table_batting.to_csv(file_name2, index=False)
    table_bowling.to_csv(file_name3, index=False)
    table_player.to_csv(file_name4, index=False)
